### Connecting with SQL Server and Importing data from SQLServer using SQLALchemy

In [1]:
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-H3D9E3H\SQLEXPRESS;"
    "DATABASE=db_churn;"
    "TrustServerCertificate=yes;"
    "Encrypt=yes;TrustServerCertificate=yes;"
)

url = (
    "mssql+pyodbc://@DESKTOP-H3D9E3H\SQLEXPRESS/db_churn"
    "?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    "&encrypt=yes&trustServerCertificate=yes"
)

In [2]:
from sqlalchemy import create_engine, text
import pandas as pd

engine = create_engine(url, connect_args={"timeout": 10})

In [7]:
#fetching vw_churn_data
churn_data = pd.read_sql("SELECT * FROM dbo.vw_churn_data", con=engine)
churn_data=pd.DataFrame(churn_data)
churn_data.head()

,Customer_ID,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,11098-MAD,Female,30,Yes,Madhya Pradesh,0,31,Deal 1,Yes,No,...,Bank Withdrawal,95.099998,6683.399902,0.00,0,631.719971,7315.120117,Stayed,Not Applicable,Not Applicable
1,11114-PUN,Male,51,No,Punjab,5,9,Deal 5,Yes,No,...,Bank Withdrawal,49.150002,169.050003,0.00,10,122.370003,301.420013,Churned,Competitor,Competitor had better devices
2,11167-WES,Female,43,Yes,West Bengal,3,28,Deal 1,Yes,Yes,...,Bank Withdrawal,116.050003,8297.500000,42.57,110,1872.979980,10237.910156,Stayed,Not Applicable,Not Applicable
3,11179-MAH,Male,35,No,Maharashtra,10,12,None,Yes,No,...,Credit Card,84.400002,5969.299805,0.00,0,219.389999,6188.689941,Stayed,Not Applicable,Not Applicable
4,11180-TAM,Male,75,Yes,Tamil Nadu,12,27,Deal 2,Yes,No,...,Credit Card,72.599998,4084.350098,0.00,140,332.079987,4556.430176,Stayed,Not Applicable,Not Applicable


In [8]:
#fetching vw_joining_data
new_data = pd.read_sql("SELECT * FROM dbo.vw_joining_data", con=engine)
new_data=pd.DataFrame(new_data)
new_data.head()

,Customer_ID,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,11751-TAM,Female,18,No,Tamil Nadu,5,7,Deal 5,No,Not Applicable,...,Mailed Check,24.299999,38.450001,0.0,0,0.000000,38.450001,Joined,Not Applicable,Not Applicable
1,12056-WES,Male,27,No,West Bengal,2,20,None,Yes,No,...,Bank Withdrawal,90.400002,268.450012,0.0,0,94.440002,362.890015,Joined,Not Applicable,Not Applicable
2,12136-RAJ,Female,25,Yes,Rajasthan,2,35,None,Yes,No,...,Bank Withdrawal,19.900000,19.900000,0.0,0,11.830000,31.730000,Joined,Not Applicable,Not Applicable
3,12257-ASS,Female,39,No,Assam,9,1,None,Yes,No,...,Credit Card,19.549999,19.549999,0.0,0,10.200000,29.750000,Joined,Not Applicable,Not Applicable
4,12340-DEL,Female,51,Yes,Delhi,0,10,None,Yes,No,...,Credit Card,62.799999,62.799999,0.0,0,42.189999,104.989998,Joined,Not Applicable,Not Applicable


In [9]:
# churn_data=pd.read_csv("/Users/ittismita/Downloads/churn_data.xls")
# churn_data= churn_data.drop(churn_data.columns[0],axis=1)
# churn_data.head()

### Exploring and Preprocessing Data

In [10]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007 entries, 0 to 6006
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Customer_ID                  6007 non-null   object 
 1   Gender                       6007 non-null   object 
 2   Age                          6007 non-null   int64  
 3   Married                      6007 non-null   object 
 4   State                        6007 non-null   object 
 5   Number_of_Referrals          6007 non-null   int64  
 6   Tenure_in_Months             6007 non-null   int64  
 7   Value_Deal                   6007 non-null   object 
 8   Phone_Service                6007 non-null   object 
 9   Multiple_Lines               6007 non-null   object 
 10  Internet_Service             6007 non-null   object 
 11  Internet_Type                6007 non-null   object 
 12  Online_Security              6007 non-null   object 
 13  Online_Backup     

In [12]:
#replacing null values in Value_Deal column
churn_data_=churn_data.copy()
churn_data_["Value_Deal"]=churn_data["Value_Deal"].replace("None","No Deal")
churn_data_["Value_Deal"].unique()

array(['Deal 1', 'Deal 5', 'No Deal', 'Deal 2', 'Deal 3', 'Deal 4'],
      dtype=object)

In [13]:
#replacing all "Not applicable" in categorical columns to "No"
binary_cat_cols=[
 'Married',
 'Phone_Service',
 'Multiple_Lines',
 'Internet_Service',
 'Online_Security',
 'Online_Backup',
 'Device_Protection_Plan',
 'Premium_Support',
 'Streaming_TV',
 'Streaming_Movies',
 'Streaming_Music',
 'Unlimited_Data',
 'Paperless_Billing']
churn_data_[binary_cat_cols]=churn_data_[binary_cat_cols].replace("Not Applicable", "No")

In [14]:
churn_data_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007 entries, 0 to 6006
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Customer_ID                  6007 non-null   object 
 1   Gender                       6007 non-null   object 
 2   Age                          6007 non-null   int64  
 3   Married                      6007 non-null   object 
 4   State                        6007 non-null   object 
 5   Number_of_Referrals          6007 non-null   int64  
 6   Tenure_in_Months             6007 non-null   int64  
 7   Value_Deal                   6007 non-null   object 
 8   Phone_Service                6007 non-null   object 
 9   Multiple_Lines               6007 non-null   object 
 10  Internet_Service             6007 non-null   object 
 11  Internet_Type                6007 non-null   object 
 12  Online_Security              6007 non-null   object 
 13  Online_Backup     

In [15]:
#mapping gender, other binary categorical columns manually
churn_data_["Gender"]=churn_data_["Gender"].map({"Female": 0, "Male": 1}).astype(int)

for col in binary_cat_cols:
    churn_data_[col]=churn_data_[col].map({"Yes":1,"No":0}).astype(int)

In [16]:
churn_data_["Customer_Status"]=churn_data_["Customer_Status"].map({"Stayed":0,"Churned":1})
churn_data_["Customer_Status"]

0       0
1       1
2       0
3       0
4       0
       ..
6002    0
6003    0
6004    1
6005    0
6006    0
Name: Customer_Status, Length: 6007, dtype: int64

### Model Training and Evaluation

In [17]:
from sklearn.model_selection import train_test_split
X=churn_data_.drop(columns=["Customer_ID","Churn_Category","Churn_Reason","Total_Revenue","Total_Charges","Customer_Status"],axis=1)
y=churn_data_["Customer_Status"]

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
X_train.columns,len(X_test.columns), len(y_train), len(y_test) 

(Index(['Gender', 'Age', 'Married', 'State', 'Number_of_Referrals',
        'Tenure_in_Months', 'Value_Deal', 'Phone_Service', 'Multiple_Lines',
        'Internet_Service', 'Internet_Type', 'Online_Security', 'Online_Backup',
        'Device_Protection_Plan', 'Premium_Support', 'Streaming_TV',
        'Streaming_Movies', 'Streaming_Music', 'Unlimited_Data', 'Contract',
        'Paperless_Billing', 'Payment_Method', 'Monthly_Charge',
        'Total_Refunds', 'Total_Extra_Data_Charges',
        'Total_Long_Distance_Charges'],
       dtype='object'),
 26,
 4805,
 1202)

In [18]:
#categorical columns
cat_cols=[i for i in X.columns if churn_data_[i].dtype=="object"]

#numeric columns
num_cols=[i for i in X.columns if churn_data_[i].dtype in ["int64","float64"]]

cat_cols,num_cols

(['State', 'Value_Deal', 'Internet_Type', 'Contract', 'Payment_Method'],
 ['Gender',
  'Age',
  'Married',
  'Number_of_Referrals',
  'Tenure_in_Months',
  'Phone_Service',
  'Multiple_Lines',
  'Internet_Service',
  'Online_Security',
  'Online_Backup',
  'Device_Protection_Plan',
  'Premium_Support',
  'Streaming_TV',
  'Streaming_Movies',
  'Streaming_Music',
  'Unlimited_Data',
  'Paperless_Billing',
  'Monthly_Charge',
  'Total_Refunds',
  'Total_Extra_Data_Charges',
  'Total_Long_Distance_Charges'])

In [19]:
#ordinal encoded column - Contract
#one hot encoded columns - remaining cat columns

ord_enc_col=["Contract"]
ohe_col=["State", "Value_Deal", "Internet_Type", "Payment_Method"]

#defining the order for ordinal encoding
contract_order = [["Month-to-Month", "One Year", "Two Year"]]

In [20]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#preprocessor for Logistic regression, scaling sensitive
preprocessor_lr=ColumnTransformer(transformers=[
    ("oheEncode",OneHotEncoder(handle_unknown="ignore"),ohe_col),
    ("ordEncode",OrdinalEncoder(categories=contract_order),ord_enc_col),
    ("scaler",StandardScaler(),num_cols)])

#preprocessor for other, scaling insensitive
preprocessor=ColumnTransformer(transformers=[
    ("oheEncode",OneHotEncoder(handle_unknown="ignore"),ohe_col),
    ("ordEncode",OrdinalEncoder(categories=contract_order),ord_enc_col),
    ("num", "passthrough", num_cols)])

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

lr_pipeline=Pipeline(steps=[("preprocessing_lr",preprocessor_lr),
                           ("lr_model", LogisticRegression(max_iter=1000))])


rf_pipeline=Pipeline(steps=[("preprocessing_rf",preprocessor),
                           ("rf_model",RandomForestClassifier(n_estimators=200, random_state=42))])


xg_pipeline=Pipeline(steps=[("preprocessing_xg",preprocessor),
                           ("xg_model",XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=5,
        random_state=42, use_label_encoder=False, eval_metric="logloss"))])


In [24]:
#training the models
from sklearn.metrics import classification_report, roc_auc_score

for name, model in [("Logistic Regression", lr_pipeline),("Random Forest", rf_pipeline),("XGBoost", xg_pipeline)]:
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    print(f"\n{name}")
    print(classification_report(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_proba))

    


Logistic Regression
              precision    recall  f1-score   support

           0       0.85      0.90      0.88       855
           1       0.72      0.62      0.67       347

    accuracy                           0.82      1202
   macro avg       0.79      0.76      0.77      1202
weighted avg       0.82      0.82      0.82      1202

AUC: 0.8665015083337546

Random Forest
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       855
           1       0.81      0.62      0.70       347

    accuracy                           0.85      1202
   macro avg       0.83      0.78      0.80      1202
weighted avg       0.84      0.85      0.84      1202

AUC: 0.8804523315974856


C:\Users\HP\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:14:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       855
           1       0.73      0.64      0.68       347

    accuracy                           0.83      1202
   macro avg       0.80      0.77      0.78      1202
weighted avg       0.82      0.83      0.82      1202

AUC: 0.8779614742909146


### Predicting on New Data - Joining data

In [25]:
#predicting on new data
# new_data=pd.read_csv("/Users/ittismita/Downloads/joining_data.xls")
# new_data.head()

In [27]:
#processing certain columns before encoding
new_data["Value_Deal"]=new_data["Value_Deal"].replace("None","No Deal")
new_data["Value_Deal"].unique()

array(['Deal 5', 'No Deal'], dtype=object)

In [28]:
#replacing all "Not applicable" in categorical columns to "No"
new_data[binary_cat_cols]=new_data[binary_cat_cols].replace("Not Applicable", "No")

In [29]:
original_data=new_data.copy()
#retain customer ids
customer_ids=original_data["Customer_ID"]

new_data=new_data.drop(columns=["Customer_ID","Churn_Category","Churn_Reason","Total_Revenue","Total_Charges","Customer_Status"],axis=1)

In [30]:
#mapping gender, other binary categorical columns manually
new_data["Gender"]=new_data["Gender"].map({"Female": 0, "Male": 1}).astype(int)

for col in binary_cat_cols:
    new_data[col]=new_data[col].map({"Yes":1,"No":0}).astype(int)

In [31]:
lr_new_preds=lr_pipeline.predict(new_data)
rf_new_preds=rf_pipeline.predict(new_data)
xg_new_preds=xg_pipeline.predict(new_data)

In [32]:
original_data["LR_Customer_Status_Predictions"]=lr_new_preds
original_data["RF_Customer_Status_Predictions"]=rf_new_preds
original_data["XG_Customer_Status_Predictions"]=xg_new_preds

In [33]:
original_data.to_csv("predicted_data.csv", index=False)